In [1]:
# ライブラリのインポート
import tensorflow as tf
import tflearn
import numpy as np
import tflearn.datasets.mnist as mnist

# CNNの作成、学習に必要なライブラリのインポート
from tflearn.layers.core import input_data, dropout, fully_connected
from tflearn.layers.conv import conv_2d, max_pool_2d
from tflearn.layers.normalization import local_response_normalization
from tflearn.layers.estimator import regression

curses is not supported on this machine (please install/reinstall curses for an optimal experience)


In [2]:
# MNISTデータセットの読み込み(ラベルをone-hot表現に変換)
x_trains, y_trains, x_tests, y_tests = mnist.load_data('data', one_hot=True)

Extracting data\train-images-idx3-ubyte.gz
Extracting data\train-labels-idx1-ubyte.gz
Extracting data\t10k-images-idx3-ubyte.gz
Extracting data\t10k-labels-idx1-ubyte.gz


In [3]:
# 読み込み直後の各データの構造
print(x_trains.shape) # (55000, 784)
print(y_trains.shape) # (55000, 10)
print(x_tests.shape)  # (10000, 784)
print(y_tests.shape)  #  (10000, 10)

(55000, 784)
(55000, 10)
(10000, 784)
(10000, 10)


In [4]:
# 画像データの前処理

# (55000,784)の2階テンソルを(None,28,28,1)の4階テンソルに変換
x_trains = x_trains.reshape([-1, 28, 28, 1])
# (10000, 784)の2階テンソルを10000×28×28×1の4階テンソルに変換
x_tests = x_tests.reshape([-1, 28, 28, 1])

In [5]:
# データの前処理後の画像データの形状
print(x_trains.shape) # (55000, 28, 28, 1)
print(x_tests.shape)  # (10000, 28, 28, 1)

(55000, 28, 28, 1)
(10000, 28, 28, 1)


In [6]:
# 正解ラベルの一部を出力
print(y_trains[0])

[0. 0. 0. 0. 0. 0. 0. 1. 0. 0.]


In [7]:
# 畳み込みニューラルネットワーク

# 初期化
tf.reset_default_graph()

# 入力層
net = input_data(shape=[None, 28, 28, 1])

# 畳み込み層1(ニューロン数：32)
net = conv_2d(incoming=net,       # 直前の層を設定
              nb_filter=32,       # フィルターの数
              filter_size=(5, 5), # フィルターのサイズ
              activation='relu'   # 活性化関数はReLU
             )

# 畳み込み層2(ニューロン数：64)
net = conv_2d(incoming=net,       # 直前の層を設定
              nb_filter=64,       # フィルターの数
              filter_size=(5, 5), # フィルターのサイズ
              activation='relu'   # 活性化関数はReLU
             )

# 全結合層(ニューロン数：128)
net = fully_connected(incoming=net,    # 直前の層を設定 
                      n_units=128,     # ニューロンの数
                      activation='relu'# 活性化関数はReLU
                     )

# 出力層の作成(ニューロン数：10)
net = fully_connected(incoming=net,
                      n_units=10,
                      activation='softmax'
                      )

# 学習条件の設定
net = tflearn.regression(
    incoming=net,                  # 対象のモデルを設定
    optimizer='adam',              # 最適化の手法はAdam
    learning_rate=0.001,           # 学習率
    loss='categorical_crossentropy'# 誤差関数は交差エントロピー
)

Instructions for updating:
Use tf.initializers.variance_scaling instead with distribution=uniform to get equivalent behavior.
Instructions for updating:
keep_dims is deprecated, use keepdims instead


In [8]:
# 学習の実行
model = tflearn.DNN(net)      # 作成したCNNと学習条件をセット
model.fit(x_trains,           # 訓練データ(画像)
          y_trains,           # 訓練データ(ラベル)
          n_epoch=10,         # 学習回数
          batch_size=32,      # ミニバッチのサイズ
          validation_set=0.2, # 訓練データの20%をテストデータに使用
          show_metric=True    # 各ステップごとに精度を出力
         )

Training Step: 13749  | total loss: 0.21946 | time: 396.147s
| Adam | epoch: 010 | loss: 0.21946 - acc: 0.9880 -- iter: 43968/44000
Training Step: 13750  | total loss: 0.19774 | time: 429.009s
| Adam | epoch: 010 | loss: 0.19774 - acc: 0.9892 | val_loss: 0.06589 - val_acc: 0.9845 -- iter: 44000/44000
--
